<a href="https://colab.research.google.com/github/komazawa-deep-learning/komazawa-deep-learning.github.io/blob/master/2021notebooks/2021_0930viola_jones_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ヴィオラ=ジョーンズ アルゴリズム (従来手法) による顔認識実験

- author: 浅川伸一
- date: 2021_0930
- filename: 2021_0930viola_jones.ipynb
- licence: MIT license
- original dataset: http://www.ai.mit.edu/courses/6.899/lectures/faces.tar.gz

In [ ]:
# ライブラリの読み込み
!git clone https://github.com/Simon-Hohberg/Viola-Jones.git
!mv Viola-Jones/violajones .

# データの読み込み
# 元データは http://www.ai.mit.edu/courses/6.899/lectures/faces.tar.gz
# 授業用にデータ数を少なくしてあります
!wget https://komazawa-deep-learning.github.io/2021komazawa_faces.tgz -O 2021komazawa_faces.tgz
!tar xzf 2021komazawa_faces.tgz

In [ ]:
import violajones.IntegralImage as integralimage
import violajones.AdaBoost as adaboost
#import violajones.Utils as utils
from IPython.core.display import Image, display

In [ ]:
# test/non-face のデータが 23k もあって時間がかかるので，0.1 倍にした
pos_training_path = 'data/train/face'
neg_training_path = 'data/train/non-face'
pos_testing_path = 'data/test/face'
neg_testing_path = 'data/test/non-face'

num_classifiers = 20
# For performance reasons restricting feature size
min_feature_height = 4
max_feature_height = 10
min_feature_width = 4
max_feature_width = 10

import os
import numpy as np
from PIL import Image

def _load_images(path):
    images = []
    fnames = []
    for _file in os.listdir(path):
        if _file.endswith('.png'):
            fnames.append(_file)
            img_arr = np.array(Image.open((os.path.join(path, _file))), dtype=float)
            img_arr /= img_arr.max()
            images.append(img_arr)
    return images, fnames

print('訓練用顔画像の読み込み...', end="")
faces_training, train_pos_fnames = _load_images(pos_training_path)
faces_ii_training = list(map(integralimage.to_integral_image, faces_training))
print(f'..done. {len(faces_training)} 枚の顔画像を読み込みました.')
print('訓練用非顔画像の読み込み...', end="")
non_faces_training, train_neg_fnames = _load_images(neg_training_path)
non_faces_ii_training = list(map(integralimage.to_integral_image, non_faces_training))
print(f'..done. {len(non_faces_training)} 枚の非顔画像を読み込みました.')

print('検証用顔画像の読み込み..', end="")
faces_testing, test_pos_fnames = _load_images(pos_testing_path)
faces_ii_testing = list(map(integralimage.to_integral_image, faces_testing))
print(f'..done. {len(faces_testing)} 枚の顔画像を読み込みました.')
print('検証用非顔画像の読み込み..', end="")
non_faces_testing, test_neg_fnames = _load_images(neg_testing_path)
non_faces_ii_testing = list(map(integralimage.to_integral_image, non_faces_testing))
print(f'..done. {len(non_faces_testing)} 枚の非顔画像を読み込みました.')    

##  Haar 特徴を用いた Ada-boost による分類器の学習

In [ ]:
# このセルを実行すると時間がかかります 20−30 分くらい
classifiers = adaboost.learn(faces_ii_training, non_faces_ii_training, 
                             num_classifiers, 
                             min_feature_height, max_feature_height, 
                             min_feature_width, max_feature_width)

## 検証

In [ ]:
print('Testing selected classifiers..', end="")
correct_faces = sum(utils.ensemble_vote_all(faces_ii_testing, classifiers))
correct_non_faces = len(non_faces_testing) - sum(utils.ensemble_vote_all(non_faces_ii_testing, classifiers))
print('..done.)

print('Result:')
print(f'  顔: {correct_faces:>5d}/{len(faces_testing):>5d}',
      f'({float(correct_faces)/len(faces_testing) * 100:.3f}%)')
print(f' 非顔: {correct_non_faces:>5d}/{len(non_faces_testing):>5d}',
      f'({float(correct_non_faces)/len(non_faces_testing) * 100:.3f}%)')

tp = correct_faces
fp = len(non_faces_testing) - correct_non_faces
fn = len(face_testing) - correct_faces
precision = tp / (tp + fp)
recall = tp / (tp + fp)
f1 = 1/(1/precision + 1/recall))

print(f'精度: {precision:.3f}')
print(f'再現率: {recall:.3f}')
print(f'F1 値: {f1:.3f}')



# 精度，再現率，F 値 の計算

精度 (precision), 再現率 (recall), F-1 値 を計算します

- 精度は，正しいの正例数 true positive と 偽の正例数 false positive との比 $tp / (tp + fp)$ で表されます。
- 再現率は，正しい正例数 を 正しい正例数と誤った負例値 の和で除したものです $tp/(tp+fn)$
- F1 値は，精度と再現率の調和平均です

- [Wikipedia entry for the Precision and recall](https://en.wikipedia.org/wiki/Precision_and_recall)
- [Wikipedia entry for the F1-score](https://en.wikipedia.org/wiki/F1_score)
